# **🐅 [Jira Tiger](https://github.com/hugozanini/jira-tiger.git)**
#### `T`ickets `I`nsights `G`eneration and `E`fficient `R`eporting

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/drive/1klKQdA3u-rJPtrqB_Qjs39hj1sWc5VbB?usp=sharing">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/hugozanini/jira-tiger">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

### ⚙️ Environment Setup

#### Cloning and repo and installing the required libraries.
**Run the following two cells just once to setup the environment.**

*After the execution it is needed to restart the sesion.*

In [ ]:
!git clone https://github.com/hugozanini/jira-tiger.git

In [ ]:
%cd jira-tiger/
!pip install -e .

#### Import the required libraries

In [ ]:
from crewai import Agent, Task, Crew, LLM, Process
from src.agents.tools.data_extraction import JiraDataExtraction
from src.agents.tools.data_processing import JiraDataProcessing
from src.agents.tools.data_analysis import ListJiraReports, ReadJiraReport, SaveJiraData, JsonFileOperations
from src.agents.tools.slack_message import SlackMessage
import yaml
import os
import json
import glob
import pandas as pd
from datetime import datetime
from crewai.tools import tool
from IPython.display import Markdown

from pathlib import Path
from typing import Tuple
from google.colab import userdata

Load secrets as environment variables

In [ ]:
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["JIRA_API_TOKEN"] = userdata.get('JIRA_API_TOKEN')
os.environ["JIRA_URL"] = userdata.get('JIRA_URL')
os.environ["JIRA_USERNAME"] = userdata.get('JIRA_USERNAME')
os.environ["SLACK_BOT_TOKEN"] = userdata.get('SLACK_BOT_TOKEN')

#### 🤖 Model selection
*LLM to be used by the agents*

In [ ]:
llm = LLM(model = 'gemini/gemini-2.0-flash-lite-preview-02-05')

### ⚙️ Project Setup

*Project-specific configurations.*

In [ ]:
# Define start and end dates to be analyzed
start_date = "2025-02-18"
end_date = "2025-02-25"

# Construct path for markdown and json files using the date range
md_path = "teams-markdown/" + start_date + "_to_" + end_date
json_path = "teams_json/" + start_date + "_to_" + end_date

# Channel ID where the slack message will be sent
channel_id = "C0882AYL4H4"

### 🚣‍♂️ Crew: Data Extraction and consolidation

#### 👨‍💻 Analytics Engineer agent

📝 **Tasks**: Data Ingestion and Data Processing

In [ ]:
analytics_engineer_agent = Agent(
    role="Senior Jira Analytics Engineer",
    goal="""
    Transform and optimize Jira project data into structured, analysis-ready formats while ensuring data quality,
    completeness, and compliance with best practices for project analyst consumption.
    """,
    backstory="""
    You are a specialized Data Integration Engineer with over 10 years of experience in Jira data processing and analytics.
    Your core expertise includes:

    Technical Skills:
    - Advanced Jira API integration and data extraction
    - Data cleaning, transformation, and validation
    - CSV and Markdown report generation
    - Data quality assurance and validation

    Domain Knowledge:
    - Deep understanding of Jira data structures and relationships
    - Expertise in agile project management metrics
    - Strong background in data documentation and reporting

    Best Practices:
    - Implements robust error handling and data validation
    - Ensures data consistency and standardization
    - Maintains clear documentation and audit trails
    - Follows data privacy and security guidelines

    Your primary focus is on delivering high-quality, actionable data that enables project analysts
    to make informed decisions and generate valuable insights from Jira project information.
    """,
    verbose=True,
    llm=llm,
    allow_delegation=False,  # Added to ensure direct handling of sensitive data
    embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }
)


##### 📝 Data Ingestion Task

In [ ]:
data_ingestion_task = Task(
    description="""
    Extract and process Jira project data to create a structured CSV file containing the board overview.

    Key Objectives:
    1. Connect to Jira and extract project board data
    2. Generate a clean, well-structured CSV file
    3. Ensure all relevant project information is captured

    Required Action:
    Use the ingest_board_overview() action from the JiraDataExtraction tool with these mandatory parameters:
    - project_id: {project_id} (Jira project identifier)
    - labels: {labels} (List of labels to filter issues)

    Success Criteria:
    - Successfully connected to Jira API
    - Data extracted without errors
    - CSV file created with proper formatting
    - All specified labels included in the extraction
    - Data properly sanitized and structured

    Output Format:
    The CSV file should contain:
    - One row per issue/item
    - Consistent date formats
    - No missing or corrupted data
    - UTF-8 encoding

    """,
    expected_output="Full path to the generated CSV file containing the Jira board overview data",
    agent=analytics_engineer_agent,
    tools=[JiraDataExtraction()]
)


##### 📝 Data Processing Task

In [ ]:
data_processing_task = Task(
    description="""
    Convert the Jira CSV data into individual team-specific markdown reports for the project analyst's review.

    Key Objectives:
    1. Filter Jira updates between {start_date} and {end_date}
    2. Generate separate markdown reports for each team
    3. Ensure reports contain only relevant updates within the specified date range

    Required Action:
    Use the create_teams_markdowns() action from the JiraDataProcessing tool with these parameters:
    - csv_file: Path to the previously generated CSV file
    - start_date: {start_date} (format: YYYY-MM-DD)
    - end_date: {end_date} (format: YYYY-MM-DD)

    Success Criteria:
    - Each team should have its own markdown report
    - Reports should only include updates within the specified date range
    - Reports should be properly formatted for analyst review
    - All markdown files should be saved in an organized directory structure

    Note: Ensure the CSV file exists and is accessible before processing.
    """,
    expected_output="Directory path containing the generated markdown reports",
    agent=analytics_engineer_agent,
    tools=[JiraDataProcessing()],
    context=[data_ingestion_task]  # This task will wait for data_ingestion_task to complete

)

#### 🔨 Creating and running crew

In [ ]:
# Creating crew
jira_crew = Crew(
    agents=[analytics_engineer_agent],
    tasks=[data_ingestion_task, data_processing_task],
    verbose=True,
    memory=False,
    process=Process.sequential,
    embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }
  )

In [ ]:
inputs = {
    'project_id': "14130",
    'labels': ["data-platform-refactoring"],
    'start_date': start_date,
    'end_date': end_date
}

In [ ]:
result = jira_crew.kickoff(inputs=inputs)

#### 🔎 Human in the loop
Look into the Markdowns to check everything was saved properly

# 🚣‍♂️ Crew : Project updates analysis

### 👨‍💻 Project Analyst Agent

📝 **Tasks**: follow-ups generation, follow-ups consolidation, and report summary

In [ ]:
project_analyst = Agent(
    role="Senior Project Performance Analyst",
    goal="""
    Deliver comprehensive, data-driven analysis of team performance through Jira metrics, providing actionable insights
    and maintaining effective communication channels with development teams to drive continuous improvement.
    """,
    backstory="""
    You are a seasoned Project Performance Analyst with 10+ years of experience in agile environments, specializing in
    team productivity analysis and process optimization.

    Technical Expertise:
    - Advanced analysis of Jira metrics and team performance indicators
    - Deep understanding of agile development workflows and metrics
    - Expert in interpreting issue relationships and dependencies
    - Proficient in identifying patterns and trends in project data

    Analytical Skills:
    - Strategic issue analysis and root cause identification
    - Data-driven decision-making and recommendation formulation
    - Sprint performance evaluation and optimization
    - Risk assessment and mitigation strategy development

    Communication Excellence:
    - Diplomatic and effective feedback delivery
    - Clear and concise reporting style
    - Engaging and positive communication approach
    - Ability to maintain professional relationships while ensuring accountability

    Personal Attributes:
    - Known for combining professionalism with approachability
    - Maintains a positive, solution-focused mindset
    - Demonstrates emotional intelligence in team interactions
    - Balances humor with professionalism in communications
    - Expert at delivering constructive feedback without creating tension

    Best Practices:
    - Ensures all analyses are backed by concrete data
    - Maintains confidentiality and data security
    - Provides context-aware recommendations
    - Follows up systematically while maintaining positive team dynamics
    - Creates actionable, specific, and measurable improvement plans
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    allow_code_execution=False,
    embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }
)


##### 📝 Follow up generation task

In [ ]:
fup_generation_template = """
    ## Task Overview
    Analyze the Jira issues report for the {team} and generate structured follow-ups between {start_date} and {end_date}.


    ## Phase 1: Sequential Report Processing
    1. Read report using read_report() from ReadJiraReport tool:
       ```
       team_data = read_report({team_report_file})


    ## Phase 2: Process the report according to the following structure:
    ```
    ## Team name: <team>
    ### Points of Contact
    [Contact list]
    ### Updated Issues
    [Recent updates]
    ### Not Updated Issues
    [Pending updates]
    ### This ends all the issues from the team <team> ###

    ## Phase 3: Analysis Workflow
    For each team report:

    1. EXTRACT CORE DATA
       * Team name (exact match)
       * Points of Contact list
       * Updated issues section
       * Non-updated issues section
       * Parent-child issue mappings

    2. PERFORM ISSUE ANALYSIS
       For each parent issue:
       * Analyze issue description
       * Review all comments
       * Examine child issues
       * Generate follow-up addressing:
         * Current progress
         * Blocking issues
         * Required clarifications
         * Specific action items

    3. GENERATE JSON OUTPUT
       Create a structured dictionary:
       * Team Information:
         * name: Exact team name
         * contacts: [@firstnamelastname format] #Example: John Doe -> @johndoe

       * Issue Arrays:
         * updated_issues: Recent activity
         * no_update_issues: Pending updates

       * Issue Details:
         * id: "DBPD-737"
         * title: Complete issue title
         * url: "https://company.atlassian.net/browse/DBPD-737"
         * workstream: Specific workstream
         * fup: Contextual follow-up comment


     4. SAVE & VERIFY OUTPUT
         Use the tool save_data, from the SaveJiraData tool to save the json data.

       * Execute save_data:
         ```
         save_data(
             data_dict=team_data, #In the dictionary format
             file_name= {team}.json",
             base_path = teams_json
             folder= {start_date}_to_{end_date}"
         )
         ```
       # Verify:
         * To guarantee the report was saved properly, guaratnte you are seeing the message "Data successfully saved to filepath.json"

     ## Quality Guidelines
           * Write professional, friendly communications
           * Use @firstnamelastname format for mentions. Example: John Doe -> @johndoe
           * Provide specific, actionable feedback
           * Reference relevant context and updates
           * Keep messages short and focused
           * Use appropriate corporate humor (relaxed but professional)
           * Include emojis for better readability
           * Bold critical information
           * Use bullet points for organization
           * Formatting Rules:
                * Always use "\n" for line breaks
                * Always use "•" for bullet points
                * Always bold titles with *
                * Always include relevant emojis
"""

In [ ]:
fup_generation_output_template = """
Save the team JSON on the following format:
{{
    "name": "Exact Team Name",
    "contacts": ["@firstnamelastname", "@firstnamelastname"],
    "updated_issues": [
        {{
            "id": "ISSUE-KEY",
            "title": "Exact Issue Title",
            "url": "https://company.atlassian.net/browse/ISSUE-KEY",
            "workstream": "Exact Workstream Name",
            "fup": "Context-specific follow-up with @firstnamelastname mentions when needed"
        }}
    ],
    "no_update_issues": [
        {{
            "id": "ISSUE-KEY",
            "title": "Exact Issue Title",
            "url": "https://company.atlassian.net/browse/ISSUE-KEY",
            "workstream": "Exact Workstream Name",
            "fup": "Context-specific follow-up with @firstnamelastname mentions when needed"
        }}
    ]
}}
"""

In [ ]:
def get_team_reports_dict(md_path):
    # Initialize empty dictionary
    team_reports = {}

    # List all files in the directory
    for filename in os.listdir(md_path):
        # Check if file is a markdown file
        if filename.endswith('.md'):
            # Extract team name by removing both '_report' and '.md' from filename
            team_name = filename.replace('.md', '')
            team_name = team_name.replace('._', ' ')
            # Create full file path
            file_path = os.path.join(md_path, filename)
            # Add to dictionary
            team_reports[team_name] = file_path

    return team_reports

In [ ]:
team_reports = get_team_reports_dict(md_path)

In [ ]:
fup_tasks = [
    Task(
        description=fup_generation_template.format(
            team=team_name,
            team_report_file=report_file,
            base_path=json_path,
            start_date = start_date,
            end_date = end_date
        ),
        expected_output=fup_generation_output_template,
        agent=project_analyst,
        tools = [ReadJiraReport(), SaveJiraData()],
    )
    for team_name, report_file in team_reports.items()
]

##### 📝 Follow up consolidation task

In [ ]:
fups_consolidation_task = Task(
    description="""
    ## Task Overview
    Consolidate all team-specific JSON reports into a single comprehensive JSON file for the period {start_date} to {end_date}.

    ## Objective
    Create a unified report that combines all individual team JSONs while maintaining data integrity and structure.

    ## Required Action
    Use the generate_consolidated_report action from the SaveJiraData tool with these parameters:
    ```
    generate_consolidated_report(
        base_path=teams_json,  # Directory containing individual team JSONs
        folder="{start_date}_to_{end_date}"  # Target period folder
    ).
    ```

    ## Success Criteria
    1. All individual team JSONs successfully merged
    2. Consolidated file maintains original data structure
    3. No data loss during consolidation
    4. Proper file naming and location

    ## Validation Steps
    - Verify consolidated JSON exists
    - Confirm all teams are included
    - Check data integrity
    - Validate JSON format

    # These keywords must never be translated and transformed:
        - Action:
        - Thought:
        - Action Input:
        because they are part of the thinking process instead of the output.

    """,
    expected_output="""
    Path to the consolidated JSON file.
    """,
    agent=project_analyst,
    tools = [SaveJiraData()],
)

##### 📝 Report summary task

In [ ]:
report_summary_task = Task(
    description="""
    # Slack Message Generation Task

    ## Objective
    Analyze a JSON file containing a consolidated report of team progress between {start_date} and {end_date}. Generate and save a structured output JSON summarizing key accomplishments, challenges, and next steps for each workstream.

    ## Input
    - A JSON file located at `{json_path}/{start_date}_to_{end_date}.json`.
    - The input contains fields such as:
      * `start_date`, `end_date`
      * `work_evolution`
      * `workstreams_summary`
      * A list of `teams`, each with:
        - `name`
        - A list of `contacts`
        - A list of `updated_issues` (if any)
        - A list of `no_update_issues` (if any)

    ## Process
    1. Load the input JSON:
       ```
       consolidated_data = read_json(file_path=f"{json_path}/{start_date}_to_{end_date}.json")
       print("Data loaded successfully.")
       ```
       Handle errors gracefully if the file cannot be loaded.

    2. Analyze key elements:
       * Recent achievements and progress
       * Current challenges and blockers
       * Cross-team dependencies
       * Resource constraints
       * Timeline concerns

    3. Generate summaries:
       A. Workstream Summaries: For each workstream, include:
          * Accomplishments
          * Challenges
          * Action items and next steps

       B. Weekly Summary: Write a concise paragraph summarizing overall progress.

    ## Output
    Save the results as a structured JSON file at:
    ```
    output_path = f"teams_json/{start_date}_to_{end_date}/report_summary.json"
    save_json(file_path=output_path, data=<dictionary(json) generated by you> )
    ```

    ## Content Guidelines
    - Keep messages concise but specific.
    - Use a professional yet approachable tone.
    - Incorporate light corporate humor where appropriate.
    - Highlight critical information using bold text
    - Use emojis sparingly to enhance readability
    * Formatting Rules:
        * Always use "\n" for line breaks
        * Always use "•" for bullet points
        * Always bold titles with *
        * Guarantee the final json is properly formated, as indicated in the expected output, without unecessary "\n" or spaces

    # Only proceed after saving at teams_json/{start_date}_to_{end_date}/report_summary.json the dictionary generated by you.

    """,
    expected_output="""
        Example of expected output to be saved:
        {{
          "start_date": "January 5, 2025",
          "end_date": "January 12, 2025",
          "work_evolution": "The Data Platform Refactoring program has seen steady progress across both infrastructure and governance workstreams. Focus has been placed on setting a solid foundation and addressing key challenges related to security, monitoring, and initial data quality frameworks. Continued attention will be required to drive adoption of new standards and ensure seamless integration of refactored components.",
          "workstreams_summary": "*:wrench: Data Infrastructure*\n\n" +
            "• *Networking Foundation*: Initial network security policies configured, focusing on restricting access and implementing core firewall rules. VPN connection established.\n" +
            "• *Telemetry & Monitoring*: Basic Cloud Logging sink setup initiated; working on defining standardized logging formats.\n" +
            "• *Compute Optimization*: Exploration of Dataproc cluster configuration, including autoscaling, to optimize resource utilization.\n" +
            "• *Service Account Security*: Initial steps taken to audit service account usage, identifying potential security risks related to external access.\n\n" +
            "*🚧 Challenges*\n" +
            "• Defining clear standards to follow.\n" +
            "• Need to make the infrastructure secure from all sides\n\n" +
            "*➡️ Next Steps*\n" +
            "• Finalize and document core networking architecture.\n" +
            "• Fully implement Cloud Logging sink and alerts for critical infrastructure events.\n" +
            "• Define clear guidelines on service accounts.\n\n\n" +
            "*:chart_with_upwards_trend: Data Gov and Experience*\n\n" +
            "• *Governance Framework*: Initial data governance policies and standards drafted, focusing on access requirements and tagging conventions.\n" +
            "• *Data Quality*: Data reconciliation processes are being tested and data validation rules are being created.\n" +
            "• *Documentation*: Training documentation is being created to allow data scientists to use the platform.\n" +
            "• *Stakeholder Alignment*: First Discovery session with stakeholder A done. \n\n" +
            "*🚧 Challenges*\n" +
            "• Difficulty in aligning stakeholders on the data governance policy, \n" +
            "• Long implementation times for new metrics.\n\n" +
            "*➡️ Next Steps*\n" +
            "• Publish first version of Data governance Policy. All stakeholders should be aligned. \n" +
            "• Start to collect metrics to have SLO and KPIs of the data quality.\n" +
            "• Continue aligning new stakeholders."
        }}

        # Only proceed after saving at teams_json/{start_date}_to_{end_date}/report_summary.json the dictionary generated by you.
    """,
    agent=project_analyst,
    tools=[JsonFileOperations()],
    context = [fups_consolidation_task]
)

### 🔨 Creating and running crew

In [ ]:
# Creating crew
jira_crew = Crew(
    agents=[project_analyst], #, project_analyst],
    tasks=fup_tasks + [fups_consolidation_task] + [report_summary_task],
    verbose=True,
    planning=False,
    process=Process.sequential,
    embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }
)

### 🔎 Human in the loop

**Look into the Json files and follow ups**

The agents lack context on the entire project, leading to occasional inaccuracies. Manual adjustments might be necessary in the follow-ups and report summary.

# 🚣‍♂️ Crew: Slack message sending

### 👨‍💻 Communication specialist agent
Tasks: Report consolidation & Slack message sending

In [ ]:
communication_agent = Agent(
    role="Communication Integration Specialist",
    goal="""
    Ensure effective and professional communication of project updates through Slack, managing the consolidation
    and delivery of reports while maintaining consistent formatting and clarity in messaging.
    """,
    backstory="""
    You are an experienced Communication Integration Specialist with extensive expertise in corporate communications
    and technical report management.

    Technical Expertise:
    - Expert in communication systems integration
    - Proficient in JSON data handling and report consolidation
    - Advanced knowledge of Slack API and messaging protocols
    - Skilled in automated reporting systems

    Communication Skills:
    - Excellence in message formatting and presentation
    - Strong attention to detail in report consolidation
    - Expert in professional communication standards
    - Skilled at maintaining consistent messaging tone

    Professional Background:
    - 10+ years experience in technical communication
    - Proven track record in automated reporting systems
    - Specialist in data consolidation and presentation
    - Expert in corporate communication protocols

    Best Practices:
    - Ensures data accuracy in consolidated reports
    - Maintains professional formatting standards
    - Follows systematic verification procedures
    - Implements proper error handling
    - Adheres to communication security protocols
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    allow_code_execution=False,
    embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }
)

##### 📝 Report consolidation task

In [ ]:
report_consolidation_task = Task(
    description="""
    ## Task Overview
    Consolidate the teams report and summary report into a final comprehensive report for the period {start_date} to {end_date}.

    ## Objective
    Create a unified report that combines the teams report with the summary report while maintaining proper structure and data integrity.

    ## Required Action
    Use the consolidate_report action from the SlackMessage tool with these parameters:
    ```
    consolidate_report(
        report_data={{
            "teams_report_path": "teams_json/{start_date}_to_{end_date}/{start_date}_to_{end_date}.json",
            "summary_report_path": "teams_json/{start_date}_to_{end_date}/report_summary.json",
            "output_path": "teams_json/{start_date}_to_{end_date}/slack_message.json"
        }}
    )
    ```

    ## Success Criteria
    1. Teams report and summary report successfully merged
    2. All placeholders properly replaced with summary data
    3. Consolidated file maintains correct structure
    4. Output file properly generated in specified location

    ## Validation Steps
    - Verify input files exist
    - Confirm successful consolidation
    - Check data integrity
    - Validate JSON format

    # These keywords must never be translated and transformed:
        - Action:
        - Thought:
        - Action Input:
        because they are part of the thinking process instead of the output.
    """,
    expected_output="""
    Path to the consolidated report file.
    """,
    agent=communication_agent,
    tools=[SlackMessage()]
)

📝 Slack message task

In [ ]:
slack_message_task = Task(
    description="""
    ## Task Overview
    Send the consolidated report to the specified Slack channel with proper formatting and structure.

    ## Objective
    Deliver the consolidated report to Slack, ensuring proper message formatting and thread organization.

    ## Required Action
    Use the send_message action from the SlackMessage tool with these parameters:
    ```
    send_message(
        channel_id = {channel_id},
        report_file_path = teams_json/{start_date}_to_{end_date}/slack_message.json
    )
    ```

    ## Success Criteria
    1. Message successfully posted
    2. All sections properly formatted
    3. Team updates correctly threaded
    4. All links and emojis properly rendered
    5. Proper error handling if message fails

    ## Validation Steps
    - Verify report file exists
    - Confirm message delivery
    - Check message formatting
    - Validate thread structure

    # These keywords must never be translated and transformed:
        - Action:
        - Thought:
        - Action Input:
        because they are part of the thinking process instead of the output.
    """,
    expected_output="""
    Dictionary containing the Slack API responses for the main message and threaded replies.
    """,
    agent=communication_agent,
    tools=[SlackMessage()],
    context = [report_consolidation_task]
)

### 🔨 Creating and running crew

In [ ]:
# Creating crew
jira_crew = Crew(
    agents=[communication_agent],
    tasks=[report_consolidation_task, slack_message_task],
    verbose=True,
    planning=False,
    process=Process.sequential,
    embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }
)

inputs = {
    'project_id': "14130",
    'labels': ["data-platform-refactoring"],
    'start_date': start_date,
    'end_date': end_date,
    'json_path': json_path,
    'channel_id': channel_id
}